**Save and reload model:**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

In [ ]:
%load_ext tensorboard

In [ ]:
rm -rf ./logs/

In [ ]:
(xtrain,ytrain), (xtest,ytest) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
xtrain.shape, type(xtrain)

In [ ]:
classes = np.unique(ytrain, return_counts=True)
classes

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(8, 5))
axs = axs.flatten() # Flatten the 2x3 grid to easily iterate

for i in range(6):
    axs[i].imshow(xtrain[i], cmap=plt.cm.gray)
    axs[i].axis('off') # Hide axes

plt.tight_layout()
plt.show()

**Normalize images:**

In [ ]:
xtrain = xtrain / 255.0
xtest = xtest / 255.0
xtrain[1].max(), xtrain[1].min()

 **Convert the categorical labels into a one-hot encoded:**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
ytrain = encoder.fit_transform(ytrain.reshape(-1, 1))
ytest = encoder.transform(ytest.reshape(-1, 1))
ytrain[1], ytest[1]

In [ ]:
ytrain.shape

**create validation data:**

In [ ]:
xval = xtest[:5001]
yval = ytest[:5001]
xtest = xtest[5001:]
ytest = ytest[5001:]

**define the model architecture:**

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

**Tensorboard:**

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

**Train & validate:**

In [ ]:

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

history = model.fit(xtrain, ytrain, epochs=30, batch_size=32, validation_data=(xval, yval), callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir logs/fit

**Save model:**

In [ ]:
model.save('fmnist_model.keras')

**Plot accuracy & loss:**

In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

**Final test:**

In [ ]:
model.evaluate(xtest, ytest)

**Reload model and compare:**

In [ ]:
reloaded_model = tf.keras.models.load_model('fmnist_model.keras')

model.summary()
reloaded_model.summary()

In [ ]:
len(model.get_weights()), len(reloaded_model.get_weights())

In [ ]:
pred1 = model.predict(xtest[:1])
pred2 = reloaded_model.predict(xtest[:1])
pred1, pred2